# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2009-10-01	Paper Title Number 1	Journal 1	This paper is about the number 1. The number 2 is left for future work.	Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).	paper-title-number-1	http://academicpages.github.io/files/paper1.pdf
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
publications = pd.read_csv("my_pubs.csv", header=0)
publications.head()


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2014-11-02,On adaptive occupant-learning window blind and...,Building Research & Information,Occupants have a significant impact upon build...,"Gunay, H. B., O'Brien, W., Beausoleil-Morrison...",2014-11-adaptive-occ-learning-window,https://www.tandfonline.com/doi/abs/10.1080/09...
1,2014-05-01,Recursive thermal building model training usin...,"Ottawa, ON: eSim.",Simplified building models have gained in popu...,"Huchuk, B., Cruickshank, C. A., O’Brien, W., &...",2014-05-recursive-thermal-model,https://pdfs.semanticscholar.org/de1c/0259cc1a...
2,2014-05-01,The effect of input uncertainty in model-based...,"Ottawa, ON: eSim.",The adoption of model-based predictive control...,"Gunay, H., O’Brien, W., Beausoleil-Morrison, I...",2014-05-iinput-uncertainity-mpc,http://www.ibpsa.org/proceedings/eSimPapers/20...
3,2016-05-18,Model-based predictive control of office windo...,Building Research & Information,In the automation of interior window shading d...,"Huchuk, B., Gunay, H. B., O'Brien, W., & Cruic...",2016-05-model-based-predictive,https://www.tandfonline.com/doi/abs/10.1080/09...
4,2013-04-07,Preliminary results of model predictive contro...,Proceedings of the Symposium on Simulation for...,Shades in buildings are widely installed and a...,"Huchuk, B., O'Brien, W., & Cruickshank, C. A. ...",2013-04-shades-mpc,https://dl.acm.org/citation.cfm?id=2500033


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [4]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [5]:
!ls ../_publications/

14-Jul-2014-07-mpc-shades.md
2013-04-07-2013-04-shades-mpc.md
2014-05-01-2014-05-iinput-uncertainity-mpc.md
2014-05-01-2014-05-recursive-thermal-model.md
2014-11-02-2014-11-adaptive-occ-learning-window.md
2014-12-01-2014-12-shortest-prediction-horizon.md
2016-05-18-2016-05-model-based-predictive.md
2017-07-01-mpc-shades.md
2018-01-01-2018-text-mining-log-book.md
2018-04-17-2018-04-energy-comfort-building-fault.md
2018-07-01-2018-07-longitudinal-tstat-behavior.md
2019-04-01-2019-04-comparison-ml.md
2019-05-01-comparison-ml.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).